<a href="https://colab.research.google.com/github/hawkeyedatatsai/machine-learning-project-temp-/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn import set_config
set_config(display='diagram')
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, roc_auc_score 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#First choice: dataset 1 - Spanish Wine Quality

In [ ]:
filename = "/content/wines_SPA.csv"
df = pd.read_csv(filename)
print(df.info())
print(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   winery       7500 non-null   object 
 1   wine         7500 non-null   object 
 2   year         7498 non-null   object 
 3   rating       7500 non-null   float64
 4   num_reviews  7500 non-null   int64  
 5   country      7500 non-null   object 
 6   region       7500 non-null   object 
 7   price        7500 non-null   float64
 8   type         6955 non-null   object 
 9   body         6331 non-null   float64
 10  acidity      6331 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 644.7+ KB
None
(7500, 11)


,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


##1. Source of data

Data set is from [here](https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset)

##2. Brief description of data


This dataset describes several popularity and description metrics on red variants of spanish wine's quality. The datasets can be used for classification or regression tasks. The task is to predict either the quality of wine or the prices using the given data. The classes are ordered and not balanced (i.e. the quality goes from almost 5 to 4 points).

##3. What is the target?

Both the *quality of the wine* and the *prices* are available to predict using the given data, and I plan to study the latter one  because

- Prices are the most straightforward way to look at the market value of a bottle of wine.
- I feel the quality of the wine are more subjective and the quality of the data maybe biased.

##4. What does one row represent? (A person?  A business?  An event? A product?)

An overview of a specific flavor of the wine which includes its originality, flavor profile and market facts.

##5. Is this a classification or regression problem?

Since I plan to predict the prices, it is going to be a regression problem.

##6. How many features does the data have?

In [ ]:
print(f' There are {df.shape[1]-1} featues in this data ')

 There are 10 featues in this data 


##7. How many rows are in the dataset?

In [ ]:
print(f' There are {df.shape[0]} rows in this data ')

 There are 7500 rows in this data 


##8. What, if any, challenges do you foresee in cleaning, exploring, or modeling this dataset?

*   Cleaning: missing value and data type

*   Exploring: finding out correlations between winery and rating.

*   Modeling: will start with linear regression, then add random forest, then end up with comparingn regression matrix including MAE, MAPE, MSE, RMSE and R2.

#Second choice: dataset 2 - Stroke Prediction



##1. Source of data

Data set is from [here](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset)

In [ ]:
filename = "/content/healthcare-dataset-stroke-data.csv"
df2 = pd.read_csv(filename)
print(df2.info())
print(df2.shape)
df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB
None
(5110, 12)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


##2. Brief description of data

According to the World Health Organization (WHO), stroke is the 2nd leading cause of death globally, responsible for approximately 11% of total deaths.


##3. What is the target?



I intend to predict whether a patient is likely to get stroke, 1 for get stroke and 0 for not get stroke,based on the features like gender, age, various diseases, and smoking status. 

##4. What does one row represent? (A person?  A business?  An event? A product?)



Each row in the data provides relavant information about the patient.

##5. Is this a classification or regression problem?


It is going to be a classification problem since we are predicting with 1 and 0  whilst 1 means if the patient had a stroke and 0 means if not

##6. How many features does the data have?



In [ ]:
print(f' There are {df2.shape[1]-1} featues in this data ')

 There are 11 featues in this data 


##7. How many rows are in the dataset?


In [ ]:
print(f' There are {df2.shape[0]} rows in this data ')

 There are 5110 rows in this data 


##8. What, if any, challenges do you foresee in cleaning, exploring, or modeling this dataset?



*   Cleaning: missing value

*   Exploring: finding out correlations between people with stroke AND their health ststus.

*   Modeling: will start with baseline, then add logreg, then end up with confusion matrix on oserving type 1 and type 2 errors.